<p align="center">
<img src ="https://raw.githubusercontent.com/microsoft/azuredatastudio/master/src/sql/media/microsoft_logo_gray.svg?sanitize=true" width="250" align="center">
</p>
 
## Deploying Azure Cognitive Services inside SQL Server 2019 Big Data Cluster
 
This notebook walks through the process of deploying Azure Cognitive Services in SQL Server 2019 Big Data Cluster.

<span style="color:red"><font size="3">Please press the "Run Cells" button to run the notebook</font></span>

### **Prerequisites**
Ensure the following tools are installed and added to PATH before proceeding.

|Tools|Description|Installation|
|---|---|---|
|kubectl | Command-line tool for monitoring the underlying Kuberentes cluster | [Installation](https://kubernetes.io/docs/tasks/tools/install-kubectl/#install-kubectl-binary-using-native-package-management) |

### **Check dependencies**

In [44]:
import pandas,sys,os,json,time,subprocess,getpass,tempfile
pandas_version = pandas.__version__.split('.')
pandas_major = int(pandas_version[0])
pandas_minor = int(pandas_version[1])
pandas_patch = int(pandas_version[2])
if not (pandas_major > 0 or (pandas_major == 0 and pandas_minor > 24) or (pandas_major == 0 and pandas_minor == 24 and pandas_patch >= 2)):
    sys.exit('Please upgrade the Notebook dependency before you can proceed, you can do it by running the "Reinstall Notebook dependencies" command in command palette (View menu -> Command Palette…).')
def run_command(command):
    print("Executing: " + command)
    stdout = subprocess.check_output(
        command,
        stderr=subprocess.STDOUT,
        shell=True).decode("utf-8")
    print(stdout)
    return stdout
    if _exit_code != 0:
        sys.exit(f'Command execution failed with exit code: {str(_exit_code)}.\n\t{command}\n')
    print(f'Successfully executed: {command}')
    
run_command('kubectl version --client=true')

### **Define which environment to deploy to, AKS or Kubeadm**

In [45]:
cluster_type = input("Which cluster are you deploying to? (AKS or KUBEADM) (Default is KUBEADM)") or "KUBEADM"

### **Setting up the parameters**

In [52]:
label="cognitive-services-read"
deployment_file=None
if cluster_type == "KUBEADM":
    deployment_file="template_kubeadm.yaml"
else:
    deployment_file="template_aks.yaml"
image="mcr.microsoft.com/azure-cognitive-services/cognitive-services-read:latest"
container_port="5000"
service_port="5700"

The Endpoint URI value is available on the Azure portal Overview page of the corresponding Cognitive Service resource. Navigate to the Overview page, hover over the Endpoint, and a Copy to clipboard  icon will appear. Copy and use where needed.

In [47]:
endpoint_url = getpass.getpass(prompt = "Endpoint Url")

This key is used to start the container, and is available on the Azure portal's Keys page of the corresponding Cognitive Service resource. Navigate to the Keys page, and click on the Copy to clipboard  icon.

In [48]:
api_key = getpass.getpass(prompt = "API Key")

### **Setup cluster context**
Enter your kube_config and your Big Data Cluster name.

In [53]:
from sys import platform

home_dir = None
if platform == "linux" or platform == "linux2":
    home_dir = os.environ.get("HOME")
elif platform == "darwin":
    home_dir = os.environ.get("HOME")
elif platform == "win32":
    home_dir = os.environ.get("USERPROFILE")
default_config_path = os.path.join(home_dir, ".kube", "config")
kube_config = os.environ.get("KUBECONFIG")
if kube_config:
    default_config_path = kube_config
kube_config = input("Enter kube config. Default: %s" % default_config_path) or default_config_path
os.environ["KUBECONFIG"] = kube_config
print("Cluster Config Location: %s" % kube_config)
namespace = input("Enter Big data cluster name")
run_command('kubectl config set-context --current --namespace=%s' % namespace)

In [50]:
from pathlib import Path
def find_file(file_name):
    root = os.getcwd()
    file_path = None
    for filename in Path(root).rglob(file_name):
        file_path = os.path.join(root, filename)
        break
    return file_path

### **Deploy The Service**
Deploy the service in your Big Data Cluster

In [58]:
config_file = find_file(deployment_file)
file_content = None
with open(config_file) as f:
    file_content = f.read()
    file_content=file_content.replace("$LABEL", label)
    file_content=file_content.replace("$IMAGE", image)
    file_content=file_content.replace("$CONTAINER_PORT", container_port)
    file_content=file_content.replace("$BILLING_END_POINT", endpoint_url)
    file_content=file_content.replace("$API_KEY", api_key)
    file_content=file_content.replace("$PORT", service_port)

fd, path = tempfile.mkstemp()
try:
    with os.fdopen(fd, 'w') as tmp:
        tmp.write(file_content)
    run_command('kubectl apply -f "%s" -n %s' % (path, namespace))
finally:
    os.remove(path)


### **Wait for deployment to complete**

In [65]:

for i in range(1, 50):
    pod_status = run_command('kubectl get pod --selector=app=%s  -n %s -o=jsonpath="{.items[0].status.containerStatuses[0].state}" ' % (label, namespace))
    if pod_status and "running" in str(pod_status):
        break
    else:
        time.sleep(5)

### **Get service endpoint**

In [64]:


host_ip = None
port = None
if cluster_type == "KUBEADM":
    host_ip = run_command('kubectl get pod --selector=app=%s -o=jsonpath="{.items[0].status.hostIP}" -n %s' % (label, namespace))
    port = run_command('kubectl get service %s-svc -o=jsonpath="{.spec.ports[0].nodePort}" -n %s' % (label, namespace))

else:
    for i in range(1, 50):
        host_ip = run_command('kubectl get service %s-svc -o=jsonpath="{.status.loadBalancer.ingress[0].ip}" -n %s' % (label, namespace))
        port = run_command('kubectl get service %s-svc -o=jsonpath="{.spec.ports[0].port}" -n %s' % (label, namespace))
        if not host_ip:
            time.sleep(5)
        else:
            break
  
def make_clickable(val):
    return '<a href="{}">{}</a>'.format(val,val)
df = pandas.DataFrame(["http://{}:{}".format(host_ip, port)])
df.style.format(make_clickable)




Now, you can use and access the service endpoint using above url.